In [10]:
from langchain_community.document_loaders import DirectoryLoader, JSONLoader

def metadata_func(record, metadata):
    metadata.update({
        "url": record.get("url", ""),
        "title": record.get("title", ""),
        "section": record.get("section", ""),
        "chunk_index": record.get("chunk_index", ""),
    })
    return metadata

loader = DirectoryLoader(
    "./data",
    glob="attack_on_Titan_Namu_part1.jsonl",
    loader_cls=JSONLoader,
    loader_kwargs={
        "jq_schema": ".",
        "json_lines": True,
        "content_key": "text",
        "metadata_func": metadata_func,
    },
)

document_list = loader.load()


In [11]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

In [16]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='attackTitan', persist_directory="./attackTitan")


In [ ]:
# import shutil

# persist_directory = "attack_on_Titan"  # ✅ ChromaDB 저장 경로
# shutil.rmtree(persist_directory)  # ✅ 기존 ChromaDB 폴더 삭제
# print(f"✅ 기존 ChromaDB ({persist_directory}) 삭제 완료!")


✅ 기존 ChromaDB (attack_on_Titan) 삭제 완료!


In [ ]:
# from langchain_chroma import Chroma

# database = Chroma(
#     collection_name="attack_on_Titan",
#     persist_directory="./attack_on_Titan",
#     embedding_function=embedding,
# )

In [19]:
from langchain_openai import ChatOpenAI

retriever = database.as_retriever(search_kwargs={"k": 3})
llm = ChatOpenAI(model="gpt-5")

query = "에렌 크루거에 대해 알려줘"
docs = retriever.invoke(query)

context = "\n\n".join([d.page_content for d in docs])

prompt = f"""You are a helpful assistant. Use the context to answer.
질문: {query}

컨텍스트:
{context}
"""

response = llm.invoke(prompt)
response.content


'에렌 크루거는 만화/애니메이션 진격의 거인에 등장하는 인물로, 마레 정부 보안당국에 침투한 엘디아계 스파이이자 “부엉이”라는 암호명으로 활동했습니다. 진격의 거인의 계승자였으며, 자신의 정체를 밝힌 뒤 그리샤 예거를 구해 임무를 맡기고(벽 안으로 들어가 왕가의 진실을 밝히고 힘을 이어가게 함) 결국 진격의 거인을 그리샤에게 계승시킨 인물입니다.\n\n핵심 포인트\n- 본명/이명: 에렌 크루거(Eren Kruger), 부엉이\n- 소속/위장: 마레 보안 당국 고위 요원으로 위장한 엘디아 저항 조직의 중개자\n- 보유 능력: 진격의 거인(계승자) — 잠입과 정보전, 변신을 통한 전투에 능함\n- 역할: 그리샤 예거를 처형 직전에서 구출, 자신의 정체를 드러내고 임무를 부여, 진격의 거인을 계승시켜 이야기의 큰 흐름을 시작하게 함\n- 상징적 장면(컨텍스트에 근거):\n  - “정체를 드러내는 에렌 크루거”\n  - “그리샤 예거에게 임무를 맡기는 에렌/엘런 크루거”\n  - “갑옷 거인에게 백브레이커를 시전하는 엘런 크루거”로 묘사된 전투 장면도 존재함\n\n비고\n- 한국어 표기에서 ‘에렌/엘런’ 등으로 혼용되기도 합니다.\n- 그의 결단과 계승은 이후의 사건 전개(예거 가문과 진격의 거인 계보)에 결정적 영향을 미칩니다.'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

llm = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the context to answer."),
    ("human", "질문: {input}\n\n컨텍스트:\n{context}")
])

combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

ai_message = retrieval_chain.invoke({"input": query})
ai_message["answer"]


In [ ]:
# RAG: reduce hallucination by stricter retrieval + prompt
from langchain_openai import ChatOpenAI

# retriever with more candidates + MMR for diversity
retriever = database.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 24}
)

query = "아홉거인에 대해 자세하게 설명해줘"
docs = retriever.invoke(query)

# if retrieval is weak, fail fast
if len(docs) < 2:
    raise ValueError("컨텍스트가 부족합니다. 질문을 구체화하거나 k를 늘려주세요.")

context = "\n\n".join([d.page_content for d in docs])

prompt = f"""You are a helpful assistant.
Use ONLY the provided context.
If the answer is not in the context, say "정보 부족".


질문: {query}

컨텍스트:
{context}
"""

llm = ChatOpenAI(model="gpt-5-mini")
response = llm.invoke(prompt)
response.content


"다음 내용은 제공된 문맥 내에 있는 정보만을 정리한 것입니다. 그 밖의 상세한 설명은 문맥에 없습니다 — 정보 부족.\n\n- 아홉 거인(Nine Titans)에 대한 설명은 별도의 '아홉 거인' 문서를 참고하라고 되어 있습니다.\n- 계승자(또는 관련 인물)로 문맥에 나열된 항목들:\n  - 아홉 거인, 가비 브라운, 나일 도크, 다이나 프리츠, 도트 픽시스, 레온하트 씨, 로그(진격의 거인), 로드 레이스(진격의 거인), 맘몬(진격의 거인), 바리스(진격의 거인), 벽의 거인, 아홉 거인, 오거(진격의 거인)\n- 완전한 시조의 거인이 아홉 거인의 모든 힘을 사용할 수 있게 되면서 그 힘으로 모든 거인들의 경질화(갑옷화)를 해제하였다(문맥 내용). 그 결과 파라디 섬 세 방벽 내부의 초대형(벽 속) 거인들이 진격하여 방벽 주변 건물들이 무너지고 많은 사상자가 발생하기 시작했다는 기술이 있습니다.\n- 아홉 거인의 능력들이 더해지면 더욱 강력해지며, 예로 갑옷의 경질화와 전퇴의 능력, 소유자의 극한 숙련도가 합쳐진 후반부의 진격(아마 특정 거인 또는 상태)은 아홉 거인들 중 상위권 전투력으로 평가된다고 문맥에 서술되어 있습니다.\n- 추가 세부사항은 엘런 예거 문서의 8.1번 문단과 아홉 거인 문서를 참고하라고 명시되어 있습니다.\n\n위 외의 구체적 능력, 각 거인의 상세 설명 등은 문맥에 포함되어 있지 않습니다 — 정보 부족."

In [9]:
# Filter category/low-signal docs to reduce hallucination
def filter_docs(docs, min_len=50):
    filtered = []
    for d in docs:
        title = (d.metadata.get('title') or '')
        url = (d.metadata.get('url') or '')
        text = (d.page_content or '').strip()
        if title.startswith('분류:'):
            continue
        if '/w/%EB%B6%84%EB%A5%98:' in url or '/w/분류:' in url:
            continue
        if len(text) < min_len:
            continue
        filtered.append(d)
    return filtered

retriever = database.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 12, 'fetch_k': 40}
)

docs = retriever.invoke(query)
docs = filter_docs(docs)
if len(docs) < 2:
    raise ValueError('정보 부족: 관련 본문 문서를 찾지 못했습니다.')

context = '\n\n'.join([d.page_content for d in docs])

prompt = f"""You are a helpful assistant.
Use ONLY the provided context.
If the answer is not in the context, say \"정보 부족\".


질문: {query}

컨텍스트:
{context}
"""

llm = ChatOpenAI(model='gpt-4o-mini')
response = llm.invoke(prompt)
response.content


'정보 부족'